In [1]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install -U ipykernel

--2020-05-20 17:39:18--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2571184 (2.5M) [text/plain]
Saving to: ‘doctor_train.csv’

doctor_train.csv    100%[===================>]   2.45M  10.2MB/s    in 0.2s    

2020-05-20 17:39:18 (10.2 MB/s) - ‘doctor_train.csv’ saved [2571184/2571184]

--2020-05-20 17:39:21--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaitin

     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


# restart runtime here and continue running cells below!

In [1]:
import autogluon as ag
from autogluon import TabularPrediction as task
import time
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])
df_D=df_D.fillna(0)
# df_D[df_D < 0] = 0
test_df_D=test_df_D.fillna(0)
# test_df_D[test_df_D < 0] = 0

from imblearn.combine import SMOTETomek
smt = SMOTETomek(ratio='auto')
df_D_over, df_Y_over = smt.fit_resample(df_D, df['Y'])
df_D_over = pd.DataFrame(df_D_over)
test_df_D.columns = df_D_over.columns

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_D_over, df_Y_over, random_state=35, test_size=0.3)
X_train['Y'] = y_train
# df_D_over['Y'] = df_Y_over

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/imblearn/utils/deprecation.py:53: DeprecationWarning: 'ratio' is deprecated from 0.4 and will be removed in 0.6 for the estimator <class 'imblearn.combine._smote_tomek.SMOTETomek'>. Use 'sampling_strategy' instead.
  cate

In [2]:
predictor = task.fit(train_data=X_train, 
                     label='Y', 
                     eval_metric='accuracy',
                     auto_stack=True,
                     verbosity=2)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200520_174958/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200520_174958/
Train Data Rows:    33895
Train Data Columns: 52
Preprocessing data ...
Here are the first 10 unique label values in your data:  [1 0]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Feature Generator processed 33895 data points with 51 features
Original Features:
	float features: 51
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 51
	object features: 0
	int features: 0
	Data preprocessing and feature engineering runtime = 0.33s ...
AutoGluon will gauge predictive performance using evaluatio

In [4]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                    model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0           CatboostClassifier_STACKER_l1   0.970497      17.046267  1618.503926                0.131595          56.376704            1       True
1                 weighted_ensemble_k0_l2   0.970497      17.062408  1629.066148                0.016141          10.562222            2       True
2   RandomForestClassifierEntr_STACKER_l1   0.965128      19.084324  1678.503069                2.169651         116.375847            1       True
3   RandomForestClassifierGini_STACKER_l1   0.964833      19.082419  1671.204197                2.167747         109.076976            1       True
4     ExtraTreesClassifierGini_STACKER_l1   0.963121      19.091862  1626.234721                2.177190          64.107499            1       True
5     ExtraTreesClassifierEntr_STACKER_l1   0.9629

In [14]:
%%time
# y_hat = predictor.predict(test_df_D)
y_hat = predictor.predict(test_df_D)

CPU times: user 1min 44s, sys: 12.4 s, total: 1min 56s
Wall time: 1min 12s


In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))

0.9712259929785916
[[7091  214]
 [ 204 7018]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      7305
           1       0.97      0.97      0.97      7222

    accuracy                           0.97     14527
   macro avg       0.97      0.97      0.97     14527
weighted avg       0.97      0.97      0.97     14527



In [0]:
le_y = le.inverse_transform(y_hat)
output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')